In [1]:
%load_ext autoreload
%autoreload 2

# Computing Trajectories


Process, starting with a list of segments, with each segment composed of the same number of axes. 

1. For each segment, compute the minimum time to run the segment, which is the longest time required for any of the axes running at full velocity
2. Set a first guess at the constant velocity for each segment, $t_c$, which calculated without accounting for acceleration, $t_c = x/t$.
3. Compute the acceleration times, $t_a$ and $t_d$, from the time to reach the max velocity at max acceleration. 
4. Set the start and end velocities, $v_0$ and $v_1$, for each segment/axis. $v_{1_1} = v_{0_2} = (v_{c_1} + v_{c_2})/2$ 
5. Recalculate $v_c$ for each segment/axis, iteratively, to convergence. 


# Fast Change To New Velocity

Maybe a fixed-width second at the start and the end of each segment can be reserved for accceleration, and then the rest of the segment runs at a fixed velocity.

Max accelerations for the few systems I've tested are at least 2,000,000 steps/sec^2, and the max velocities are in the range of 6,000 to 15,000 steps/sec. So, a comfortable acceleration period is about 0.03 sec, around 200 steps. This technique would work for any segment longer than 0.06 sec. 

A chain of segments should (must?) have the same direction for all segments for each axis. ( Axes can go in different directions, but each segment of the axis must goin in the same direction. ) 



## Longest Axis

For the longest axis:

&nbsp;&nbsp;&nbsp;&nbsp;$v_0$ is the initial velocity<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$v_1$ is the final velocity<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$v_c = v_{max}$ is the velocity at the zero acceleration period<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$\Delta{v_0} = |v_c-v_0|$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$\Delta{v_1} = |v_c-v_1|$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$t_a = t_d = \frac{v_{max}}{a_{max}}$ Time to accelerate or decelerate<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$x_a = \frac{v_0+v_c}{2}t_a$ Steps during accel phase <br/>
&nbsp;&nbsp;&nbsp;&nbsp;$x_d = \frac{v_1+v_c}{2}t_d$ Steps during decel phase <br/>
&nbsp;&nbsp;&nbsp;&nbsp;$x_c = x - x_a - x_d$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$t_c = \frac{x_c}{v_{max}}$

if $x_c < 0$, then compute a triangular profile. 


The value for $t_c$ for the longest axis sets the value of $t_c$ for all other axes. 


## Other Axes

The calculations for the longest axes sets the value of $t_c$, so for the other axes we have to calculate $v_c$

$$\begin{array}{ll}
x_a = \frac{v_0+v_c}{2}t_a & \text{Steps during accel phase} \tag{1}\label{eq1}\\ 
x_d = \frac{v_1+v_c}{2}t_d & \text{Steps during decel phase} \\ 
x_c = x - x_a - x_d & \text{Steps during constant velocity phase} \\
v_c' = \frac{x_c}{t_c} & \text{Update constant velocity} \\
\end{array}$$

The series of equations above is iterated ( $v_c \leftarrow v_c'$ ) until the result converges, $|v_c - v_c'| < \epsilon$.


This references $\eqref{eq1}$
    


In [2]:


def solve_axis(x, t_a, t_c, t_d, v_0, v_1):
    

    v_c = x / t_c
    
    
    def calc_x(v_c):

        x_a = (v_0+v_c)*t_a/2
        x_d = (v_1+v_c)*t_d/2
        x_c = x - x_a - x_d

        return x_a, x_c, x_d


    def update_vc(v_c):
        x_a, x_c, x_d = calc_x(v_c)
        v_c_p = x_c / t_c

        return v_c_p

    for i in range(10):
        print(v_c, calc_x(v_c))
        v_c_p = update_vc(v_c)

        d = abs(v_c - v_c_p)

        v_c = v_c_p

        if i > 10 or d < 0.1:
            return (v_c,)+calc_x(v_c) # v_c, x_a, x_c, x_d

            
a_max = 500_000
v_max = 5000

x = 1000

v_0 = 500
v_1 = 2000

t = 1
t_a = t_d = v_max / a_max
t_c =  t - t_a - t_d

solve_axis(x, t_a, t_c, t_d, v_0, v_1)

1020.4081632653061 (7.6020408163265305, 977.295918367347, 15.10204081632653)
997.2407330279051 (7.486203665139526, 977.5275926697209, 14.986203665139527)
997.4771353772662 (7.487385676886332, 977.5252286462273, 14.987385676886332)


(997.4747231083952, 7.487373615541976, 977.5252527689161, 14.987373615541976)

# Acceleration, no initial velocity 

<img src="acceleration-only.png" width="400" height="300">

## Fixed total time 

$v_t$ is the velocity at time $t$ with constant acceleration $a$.

Max distance for time $t$, $x_{max} = \frac{1}{2}v_at$. But $v_t = at$, so $x_{max} = \frac{1}{2}{a}t^2$

&nbsp;&nbsp;&nbsp;&nbsp;$x_e = \frac{1}{2}at_c^2$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$x_a = \frac{1}{2}at_a^2$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$x_c = v_ct_c$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$x_{max} = \frac{1}{2}v_tt = x_a+x_c+x_e$<br/>

The actual distance traveled will be $x_a+x_c$, which is also $x_{max}-x_e$.

$x = x_{max}-x_e =  \frac{1}{2}v_tt - \frac{1}{2}at_c^2$

Solving for $t_c$
1. $x =  \frac{1}{2}at^2 - \frac{1}{2}at_c^2$
1. $2x =  at^2 - at_c^2$
1. $at^2 - 2x =  at_c^2$
1. $\frac{(at^2 - 2x)}{a} =  t_c^2$
1. $t_c = \sqrt{\frac{(at^2 - 2x)}{a}} $

The velocity at $t_c$ is $v_c=a{t-t_c}$

## Variable total time 

For the axis that moves the longest distance, maybe we should assume that $v_c$ is $v_{max}$. Then, we calculate the time required to accelerate to either the total distance, or to the top speed. 

### Constant acceleration

if $v = at < v_{max}$:

&nbsp;&nbsp;&nbsp;&nbsp;$x=\frac{1}{2}at^2$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$t = \sqrt{2x/a}$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;$v = at$<br/>

### Accelerate to $v_{max}$:

&nbsp;&nbsp;&nbsp;&nbsp;$t_a = \frac{v_{max}}{a}$ Time of acceleration</br>
&nbsp;&nbsp;&nbsp;&nbsp;$x_a = \frac{1}{2}at_a^2$ Distance at end of acceleration</br>
&nbsp;&nbsp;&nbsp;&nbsp;$x_c = x - x_a $ Distance to cover in constant velocity period</br>
&nbsp;&nbsp;&nbsp;&nbsp;$t_c = \frac{x_c}{v_{max}}$ Time for constant velocity motion</br>
&nbsp;&nbsp;&nbsp;&nbsp;$t =  \frac{x_c}{v_{max}} + \frac{v_{max}}{a}$ Total time of segment</br>

Maybe the procedure calculates both of these times, and selects the smaller of the two, as long as the velocity constraint is met. 


## Example Calculation

In [3]:
t = 10
a = 20
v_t = a*t
x_t = .5*a*t**2
v_t, x_t

(200, 1000.0)

In [4]:
t_a = 5
x_a = .5*a*t_a**2
v_c = v_a = a*t_a
t_c = t - t_a
x_c = t_c * v_c
x =  x_a + x_c
ex1 = x_a, v_c, x_c, x
ex1

(250.0, 100, 500, 750.0)

So, solving for $x = 750$ with $t=10, a=20$ should yield $t_c=t_a=5$

In [5]:
from math import sqrt
t_c_2 = sqrt( ( (a*t**2) - 2*x) / a )
assert( t_c_2 == t_c)

Using the second method, accelerating to $v_{max}$:

In [6]:
v_max = v_a # To get same results as first example calc

t_a = v_max / a
x_a = .5*a*t_c**2
x_c = x - x_a
t_c_ = x_c/v_max
t_ = (x_c/v_max) + v_max/a


#assert(t == t_)
#assert(t_c == t_c_)
#ex2 = x_a, v_max, x_c, x
#assert(ex1==ex2)
#t_, ex1

In [7]:
from trajectory.segments import * 

sl = SegmentList([Joint(5000, 100_000, 100_000), Joint(1000, 100_000, 100_000)])
sl.add_distance_segment([10000, 10000])
sl.add_distance_segment([10000, 10000])
sl.add_distance_segment([10000, 10000])
sl.add_distance_segment([10000, 5000])
sl.add_distance_segment([10000, 1000])
sl.add_distance_segment([10000, 500])
sl.add_distance_segment([10000, 0])
sl.add_distance_segment([10000, 0])

sl.update()

print(sl)

print(sl.dataframe.head())


|0.0100 10.2000 0.0000| [    5/0      9995@980        0/980  ]  [    5/0      9995@1000       0/1000 ]
|0.0000 10.2051 0.0000| [    0/980   10000@980        0/980  ]  [    0/1000  10000@1000       0/1000 ]
|0.0000 10.2000 0.0049| [    0/980    9995@980        5/1475 ]  [    0/1000   9995@1000       5/1000 ]
|0.0049 5.0592 0.0152| [    9/1475   9965@1970      26/3485 ]  [    5/1000   4980@1000      15/744  ]
|0.0152 2.0321 0.0013| [   64/3485   9931@5000       5/5000 ]  [    9/744     990@487        1/359  ]
|0.0013 2.1667 0.0012| [    6/5000   9988@5000       6/5000 ]  [    0/359     500@231        0/115  ]
|0.0012 1.9988 0.0000| [    6/5000   9994@5000       0/5000 ]  [    0/115       0@0          0/0    ]
|0.0000 1.9500 0.0500| [    0/5000   9750@5000     250/0    ]  [    0/0         0@0          0/0    ]
   index  axis        x     v_i     v_f ss      del_t             n        cn  \
0      0     0      5.0     0.0   979.9  a   0.010000  0.000000e+00  0.003054   
1      1     1     